Welcome to the Notebook for running and developing validations using the LBLOD validation tool.

We load Javascript libraries to i) run the validation tool, ii) display the JSON result, and iii) use Comunica to parse Turtle

In [1]:
import { validateDocument, fetchDocument } from '@lblod/lib-decision-validation';
import { display } from "https://deno.land/x/display/mod.ts";

import QueryEngine from '@comunica/query-sparql';
const engine = new QueryEngine.QueryEngine();

This is a help function to parse Turtle text into bindings, which is expected by the validation tool.
Turtle is used for defining the SHACL shapes and data input.

In [2]:
export async function getBindingsFromTurtle(content: string): Promise<Bindings[]> {
    const bindingsStream: BindingsStream = await engine.queryBindings(
      `
        SELECT ?s ?p ?o
        WHERE {
          ?s ?p ?o .
        }
      `,
      {
        sources: [
          {
            type: 'serialized',
            value: content,
            mediaType: 'text/turtle',
            baseIRI: 'http://example.org/',
          },
        ],
      },
    );
  
    return bindingsStream.toArray();
  }

The next block defines a SPARQL CONSTRUCT query and SPARQL endpoint URL to retrieve all triples in Turtle format from a municipality (in this case, Hooglede).

In [3]:
const bestuurseenheidUuid = 'd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb';

const bestuurseenheidQuery = `
construct {
  ?s ?p ?o .
}
where {
graph <http://mu.semte.ch/graphs/organizations/${bestuurseenheidUuid}/LoketLB-mandaatGebruiker> {
    ?s ?p ?o .
  }
}
`;
const bestuurseenheidUrl = `http://localhost:8890/sparql/?default-graph-uri=&query=${encodeURIComponent(bestuurseenheidQuery)}`;
const bestuurseenheidBindings = await fetchDocument(bestuurseenheidUrl);
console.log(bestuurseenheidUrl);

http://localhost:8890/sparql/?default-graph-uri=&query=%0Aconstruct%20%7B%0A%20%20%3Fs%20%3Fp%20%3Fo%20.%0A%7D%0Awhere%20%7B%0Agraph%20%3Chttp%3A%2F%2Fmu.semte.ch%2Fgraphs%2Forganizations%2Fd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb%2FLoketLB-mandaatGebruiker%3E%20%7B%0A%20%20%20%20%3Fs%20%3Fp%20%3Fo%20.%0A%20%20%7D%0A%7D%0A


Overview of all use cases we want to cover for LMB:

1 mandatarissen
1.1 een persoon heeft voor 1 mandaat geen overlappende mandatarissen
1.2 voor een mandataris vallen de start- en einddatums steeds binnen de 
bindingStart en bindingEinde van het bestuursorgaan
1.3 voor een orgaan valt het aantal mandatarissen op elk moment tussen 
de min en max aantallen
1.4 een mandataris heeft steeds een publicatiestatus
1.5 een mandataris heeft steeds een status
1.6 een mandataris heeft steeds een startdatum
1.7 een mandataris heeft steeds een persoon
1.8 een mandataris heeft steeds een mandaat
1.9 een mandataris in gemeenteraad, burgemeester of college van 
burgemeester en schepenen organen zit altijd in een fractie (of is 
onafhankelijk)
1.10 enkel mandatarissen van type schepen of burgemeester hebben 
beleidsdomeinen
1.11 (optioneel, faciliteitengemeenten doen dit niet): een mandataris in 
de gemeenteraad zetelt ook als lid van het ocmw
1.12 (optioneel, faciliteitengemeenten doen dit niet): een mandataris 
voorzitter van de gemeenteraad zetelt ook als voorzitter van het ocmw
1.13 (optioneel, faciliteitengemeenten doen dit niet): een mandataris in 
het college van burgemeester en schepenen zetelt ook in het vast bureau
1.14 (optioneel, faciliteitengemeenten doen dit niet): een (evt. 
aangewezen) burgemeester is ook voorzitter van het vast bureau
1.15 de voorzitter van het bcsd is ook lid van raad van maatschappelijk 
welzijn of vast bureau
1.16 de mandataris die een verhinderde mandataris vervangt heeft een 
startdatum gelijk aan de startdatum van de verhinderde mandataris
1.17 een mandataris van gemeenteraad, burgemeester en college 
burgemeester en schepenen is gebonden aan een persoon uit een kieslijst 
van de overeenkomstige periode
1.18 een mandataris met de publicatiestatus bekrachtigd heeft steeds een 
link naar een besluit (of een lmb:linkToBesluit als dat niet als linked 
data is gepubliceerd)
1.19 mandatarissen met een classificatie schepen, toegevoegde schepen of 
gemeenteraadslid hebben steeds een rangorde

1.1 een persoon heeft voor 1 mandaat geen overlappende mandatarissen

Below, we can find the SPARQL template query and SHACL shape for the 1.1 use case.
The Turtle string is converted into bindings.

In [4]:
const mandataris_1_1_query: string = `
    PREFIX besluit: <http://data.vlaanderen.be/ns/besluit#>
    PREFIX mandaat: <http://data.vlaanderen.be/ns/mandaat#>
    PREFIX org: <http://www.w3.org/ns/org#>
                
    select DISTINCT ($this as ?this) ?value
    where {
        {
            select (count(distinct(?mandataris)) as ?aantalMandatarissen)
            where {
                ?mandataris mandaat:isBestuurlijkeAliasVan $this ;
                                org:holds ?mandaat .
            }
            group by ?mandaat
        }
            
        FILTER (?aantalMandatarissen > 1)
        BIND ("Persoon heeft voor 1 mandaat overlappende/meerdere mandatarissen" as ?value)
    }`;
    
const mandataris_1_1_shape: string = `
@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/mandataris_1_1_blueprint>
  a sh:NodeShape ;
  sh:targetClass <http://www.w3.org/ns/person#Person> ;
  sh:sparql [
		sh:select """${mandataris_1_1_query}""" ;
        sh:message 'Persoon heeft voor 1 mandaat overlappende/meerdere mandatarissen'
] .`;

const mandataris_1_1_bindings: Bindings[] = await getBindingsFromTurtle(mandataris_1_1_shape);

In [ ]:
To test the shape, we retrieved a subject from the triple store that is applicable. In this case, we choose a person URI.
Then, the template query from above is applied on this subject.
Next, a Comunica URL is generated to debug the results of this validation.

In [5]:
// Test subject
const subject = 'http://data.lblod.info/id/personen/51ecbbe8e8cdd33ec1c45e69f986b4c6e88dbbcb475747faf69cea6550debb17';
const mandataris_1_1_subject_query = mandataris_1_1_query.replaceAll('$this', `<${subject}>`);

const mandataris_1_1_comunica = `https://query.linkeddatafragments.org/#datasources=${bestuurseenheidUrl}&query=${encodeURIComponent(mandataris_1_1_subject_query)}`;
console.log(mandataris_1_1_comunica);

https://query.linkeddatafragments.org/#datasources=http://localhost:8890/sparql/?default-graph-uri=&query=%0Aconstruct%20%7B%0A%20%20%3Fs%20%3Fp%20%3Fo%20.%0A%7D%0Awhere%20%7B%0Agraph%20%3Chttp%3A%2F%2Fmu.semte.ch%2Fgraphs%2Forganizations%2Fd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb%2FLoketLB-mandaatGebruiker%3E%20%7B%0A%20%20%20%20%3Fs%20%3Fp%20%3Fo%20.%0A%20%20%7D%0A%7D%0A&query=%0A%20%20%20%20PREFIX%20besluit%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fbesluit%23%3E%0A%20%20%20%20PREFIX%20mandaat%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fmandaat%23%3E%0A%20%20%20%20PREFIX%20org%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2Fns%2Forg%23%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20select%20DISTINCT%20(%3Chttp%3A%2F%2Fdata.lblod.info%2Fid%2Fpersonen%2F51ecbbe8e8cdd33ec1c45e69f986b4c6e88dbbcb475747faf69cea6550debb17%3E%20as%20%3Fthis)%20%3Fvalue%0A%20%20%20%20where%20%7B%0A%20%20%20%20%20%20%20%20%7B%0A%20%20%20%20%20%20%20%20%20%20%20%20select%2

The full validation on all persons happens below:

In [7]:
const mandataris_1_1_validationReport = await validateDocument(bestuurseenheidBindings, mandataris_1_1_bindings);
await display({ "application/json": mandataris_1_1_validationReport},{ raw: true });

1.2 voor een mandataris vallen de start- en einddatums steeds binnen de 
bindingStart en bindingEinde van het bestuursorgaan

In [35]:
const mandataris_1_2_query: string = `
    PREFIX besluit: <http://data.vlaanderen.be/ns/besluit#>
    PREFIX mandaat: <http://data.vlaanderen.be/ns/mandaat#>
    PREFIX org: <http://www.w3.org/ns/org#>
                
    select DISTINCT ($this as ?this) ?value
    where {
        $this a mandaat:Mandataris ;
            org:holds/^org:hasPost ?bestuursorgaanInTijd ;
            mandaat:start ?startMandataris .
        OPTIONAL {
            $this mandaat:einde ?eindeMandataris .
        }

        ?bestuursorgaanInTijd mandaat:bindingStart ?startBestuursorgaan ;
                              mandaat:bindingEinde ?eindeBestuursorgaan .

        BIND(strdt(substr(str(?startMandataris), 1, 10), xsd:date) as ?startMandatarisDate)
        BIND(strdt(substr(str(?eindeMandataris), 1, 10), xsd:date) as ?eindeMandatarisDate)
        BIND(strdt(substr(str(?startBestuursorgaan), 1, 10), xsd:date) as ?startBestuursorgaanDate)
        BIND(strdt(substr(str(?eindeBestuursorgaan), 1, 10), xsd:date) as ?eindeBestuursorgaanDate)
        
        BIND((?startMandatarisDate >= ?startBestuursorgaanDate && ?startMandatarisDate <= ?eindeBestuursorgaanDate) AS ?startValid)
        BIND(if(!bound(?eindeMandatarisDate), true, ?eindeMandatarisDate >= ?startBestuursorgaanDate && ?eindeMandatarisDate <= ?eindeBestuursorgaanDate) as ?eindeValid)
        
        FILTER(!?startValid || !?eindeValid)
       
        BIND ("Mandataris heeft start- en einddatum die buiten de bindingStart en bindingEinde van het bestuursorgaan vallen." as ?value)
    }`;
    
const mandataris_1_2_shape: string = `
@prefix sh: <http://www.w3.org/ns/shacl#> .

<http://example.org/mandataris_1_2_blueprint>
  a sh:NodeShape ;
  sh:targetClass <http://data.vlaanderen.be/ns/mandaat#Mandataris> ;
  sh:sparql [
		sh:select """${mandataris_1_2_query}""" ;
        sh:message 'Mandataris heeft start- en einddatum die buiten de bindingStart en bindingEinde van het bestuursorgaan vallen.'
] .`;

const mandataris_1_2_bindings: Bindings[] = await getBindingsFromTurtle(mandataris_1_2_shape);

In [36]:
// Test subject
const subject = 'http://data.lblod.info/id/mandatarissen/4a979b34-e796-4db4-b102-f2e5b9b65b4d';
const mandataris_1_2_subject_query = mandataris_1_2_query.replaceAll('$this', `<${subject}>`);

const mandataris_1_2_comunica = `https://query.linkeddatafragments.org/#datasources=${encodeURIComponent(bestuurseenheidUrl)}&query=${encodeURIComponent(mandataris_1_2_subject_query)}`;
console.log(mandataris_1_2_comunica);

https://query.linkeddatafragments.org/#datasources=http%3A%2F%2Flocalhost%3A8890%2Fsparql%2F%3Fdefault-graph-uri%3D%26query%3D%250Aconstruct%2520%257B%250A%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%257D%250Awhere%2520%257B%250Agraph%2520%253Chttp%253A%252F%252Fmu.semte.ch%252Fgraphs%252Forganizations%252Fd168033a9bac278fa744c425e078eeabd304397f953feaaf5327b4e039aecacb%252FLoketLB-mandaatGebruiker%253E%2520%257B%250A%2520%2520%2520%2520%253Fs%2520%253Fp%2520%253Fo%2520.%250A%2520%2520%257D%250A%257D%250A&query=%0A%20%20%20%20PREFIX%20besluit%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fbesluit%23%3E%0A%20%20%20%20PREFIX%20mandaat%3A%20%3Chttp%3A%2F%2Fdata.vlaanderen.be%2Fns%2Fmandaat%23%3E%0A%20%20%20%20PREFIX%20org%3A%20%3Chttp%3A%2F%2Fwww.w3.org%2Fns%2Forg%23%3E%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%0A%20%20%20%20select%20DISTINCT%20(%3Chttp%3A%2F%2Fdata.lblod.info%2Fid%2Fmandatarissen%2F4a979b34-e796-4db4-b102-f2e5b9b65b4d%3E%20as%20%3Fthis)%20%3Fvalue%0A%20%20%

In [37]:
const mandataris_1_2_validationReport = await validateDocument(bestuurseenheidBindings, mandataris_1_2_bindings);
await display({ "application/json": mandataris_1_2_validationReport},{ raw: true });